# `Wine Scraping` 🍷
***

![Image](https://img.passeportsante.net/1200x675/2021-05-03/i102200-vin.webp)

In [99]:
from bs4 import BeautifulSoup as BS
from pathlib import Path
import polars as pl
import re
from dataclasses import dataclass

In [4]:
racine = Path(".").resolve()
print(racine)

C:\Users\guill\OneDrive - Université de Tours\Bureau\M2\ML\Wine-Scraping


In [5]:
test_pages = racine / "test_pages"
print(test_pages)

C:\Users\guill\OneDrive - Université de Tours\Bureau\M2\ML\Wine-Scraping\test_pages


In [6]:
for chemin in test_pages.glob("*.html"):
    print(chemin)

C:\Users\guill\OneDrive - Université de Tours\Bureau\M2\ML\Wine-Scraping\test_pages\vin_blanc_page_1.html
C:\Users\guill\OneDrive - Université de Tours\Bureau\M2\ML\Wine-Scraping\test_pages\vin_rose_page_1.html
C:\Users\guill\OneDrive - Université de Tours\Bureau\M2\ML\Wine-Scraping\test_pages\vin_rouge_page_1.html


In [7]:
htmls = list()
for chemin in test_pages.glob("*.html"):
    htmls.append(chemin.read_text(encoding="utf8"))

In [8]:
soupe = BS(htmls[1])

# NOM / ANNEE / LIEU

In [9]:
result_items = soupe.find_all(name="div", attrs={"class": "vue-product-name"})

In [10]:
result_title = [x.text for x in result_items]

In [11]:
contenance = [x[-6:] for x in result_title]

In [46]:
contenance

['EZ\xa03 L',
 '0.75 L',
 '0.75 L',
 '0.75 L',
 '0.75 L',
 '0.75 L',
 '0.75 L',
 '0.75 L',
 '0.75 L',
 '0.75 L',
 '0.75 L',
 '0.75 L',
 '0.75 L',
 '0.75 L',
 '0.75 L',
 '0.75 L',
 '0.75 L',
 '0.75 L',
 '0.75 L',
 '0.75 L',
 '0.75 L',
 '0.75 L',
 '0.75 L',
 '0.75 L',
 '0.75 L',
 '0.75 L',
 '0.75 L',
 '0.75 L',
 '0.75 L',
 '0.75 L']

In [13]:
def extract_year(text):
    match = re.search(r'\b\d{4}\b', text)
    if match:
        return int(match.group())
    else:
        return None

In [14]:
years = [extract_year(item) for item in result_title]

In [15]:
years

[2021,
 2022,
 2022,
 2020,
 2022,
 2022,
 2022,
 2022,
 2022,
 None,
 2022,
 2022,
 2022,
 2022,
 2022,
 2022,
 2022,
 2022,
 2021,
 2022,
 2022,
 2022,
 2022,
 2022,
 2022,
 2022,
 2022,
 2022,
 2022,
 None]

# PRIX

In [104]:
result_find_price = soupe.find_all(name="div", attrs={"class": "vue-product-prices"})

In [105]:
result_price = [x.text for x in result_find_price]

In [106]:
result_price[0]

'-20%49,90\xa0€39,90\xa0€'

In [107]:
def extract_promo(text):
    match = re.search(r'-\d+%', text)
    if match:
        return match.group()
    else:
        return None

In [108]:
def extract_prix_vente(text):
    match = re.search(r'\d+,\d+', text)
    if match:
        return match.group()
    else:
        return None

In [109]:
def extract_prix_apres_promo(text):
    match = re.search(r'€(\d+,\d+)', text)
    if match:
        return match.group(1)
    else:
        return None

In [110]:
[extract_prix_apres_promo(item) for item in result_price]

['39,90',
 '6,80',
 '11,80',
 '32,90',
 '7,95',
 '5,95',
 '13,40',
 '7,40',
 '17,90',
 None,
 '4,95',
 '5,99',
 '5,20',
 '5,99',
 None,
 None,
 None,
 '11,00',
 '12,20',
 None,
 '13,50',
 '8,50',
 None,
 None,
 None,
 '7,85',
 None,
 '12,90',
 '6,10',
 None]

# c bon ou pas mon reuf ? (autrement appelé **AVIS**)
quand-est ce qu'on se descend une teille ?

In [149]:
result_find_avis = soupe.find_all(name="div", attrs={"class": "vue-avis-block"})

In [150]:
result_avis = [x.text for x in result_find_avis]

In [153]:
result_avis[10]

"16.2/20\xa0 26 notesJ'ai bu : Nadine B30/03/2023Achat confirméUn\n rosé léger, fruité, clair, avec des notes d'agrumes et de pêche, \npersonnellement, je le trouve très agréable. En apéritif, ou en \naccompagnement de grillades, il est parfait.Voir tous les commentairesLire le top avis"

In [154]:
def extract_note(text):
    match = re.search(r'(\d+)/20', text)
    if match:
        return match.group(1)
    else:
        return None

In [155]:
def extract_nb_avis(text):
    match = re.search(r'(\d+) notes', text)
    if match:
        return match.group(1)
    else:
        return None

In [156]:
[extract_nb_avis(item) for item in result_avis]

['1',
 '3',
 '2',
 None,
 '10',
 '5',
 None,
 '2',
 None,
 '47',
 '26',
 '57',
 '51',
 '37',
 '23',
 '10',
 '22',
 '5',
 None,
 '24',
 '18',
 '27',
 '16',
 '18',
 '27',
 '4',
 '3',
 '17',
 '3',
 '107']

# La soupe ça fait grandir ?

In [183]:
def extract_result(soupe):
    result_find_avis = soupe.find_all(name="div", attrs={"class": "vue-avis-block"})
    result_avis = [x.text for x in result_find_avis]
    result_find_price = soupe.find_all(name="div", attrs={"class": "vue-product-prices"})
    result_price = [x.text for x in result_find_price]
    return {
        "result_avis": result_avis,
        "result_price" : result_price
    }

# 💥 FUSIONNNNNNNNNNNNNNNN 💥

In [175]:
@dataclass
class Vin:
    prix: str
    promo: str
    prix_promo: str
    note : str
    nb_avis : str

In [176]:
def extract_vin_avis(text1, text2)-> Vin:
    prix = extract_prix_vente(text1)
    promo = extract_promo(text1)
    prix_promo = extract_prix_apres_promo(text1)
    note = extract_note(text2)
    nb_avis = extract_nb_avis(text2)
    
    return Vin(
        prix = prix,
        promo = promo,
        prix_promo=prix_promo,
        note = note,
        nb_avis = nb_avis
    )

In [186]:
[extract_vin_avis(item, item2) for (item, item2) in zip(extract_result(soupe)["result_price"], extract_result(soupe)["result_avis"])]


[Vin(prix='49,90', promo='-20%', prix_promo='39,90', note='16', nb_avis='1'),
 Vin(prix='8,60', promo='-20%', prix_promo='6,80', note='3', nb_avis='3'),
 Vin(prix='16,00', promo='-26%', prix_promo='11,80', note='5', nb_avis='2'),
 Vin(prix='40,00', promo='-17%', prix_promo='32,90', note='07', nb_avis=None),
 Vin(prix='10,70', promo='-25%', prix_promo='7,95', note='6', nb_avis='10'),
 Vin(prix='8,50', promo='-30%', prix_promo='5,95', note='6', nb_avis='5'),
 Vin(prix='14,90', promo='-10%', prix_promo='13,40', note='03', nb_avis=None),
 Vin(prix='9,90', promo='-25%', prix_promo='7,40', note='16', nb_avis='2'),
 Vin(prix='19,90', promo='-10%', prix_promo='17,90', note='03', nb_avis=None),
 Vin(prix='212,80', promo='-14%', prix_promo=None, note='7', nb_avis='47'),
 Vin(prix='7,50', promo='-34%', prix_promo='4,95', note='2', nb_avis='26'),
 Vin(prix='8,95', promo='-33%', prix_promo='5,99', note='2', nb_avis='57'),
 Vin(prix='7,50', promo='-30%', prix_promo='5,20', note='4', nb_avis='51'),
 